<a href="https://colab.research.google.com/github/Kaamraan19064/OPD/blob/master/Copy_of_AML_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from scipy.io import loadmat
import numpy as np
from matplotlib import pyplot as plt
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
import os
import shutil
import time
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torch.nn.functional as F
import pickle
from torch.utils.data import Dataset, DataLoader,TensorDataset
import math

In [4]:
test_raw=datasets.SVHN(root='datasets/SVHN/test/',split='test',  download=True)
train_raw=datasets.SVHN(root='datasets/SVHN/train/',split='train',  download=True)
extraset_raw = datasets.SVHN('datasets/SVHN/extra', split='extra', download=True)

Using downloaded and verified file: datasets/SVHN/test/test_32x32.mat
Using downloaded and verified file: datasets/SVHN/train/train_32x32.mat


In [ ]:
train_images = np.array(train_raw.data)
test_images = np.array(test_raw.data)
extra_images = np.array(extraset_raw.data)
# train_image=train_images/255
# test_images=test_images/255

train_labels = train_raw.labels
test_labels = test_raw.labels
extra_labels = extraset_raw.labels

train_labels= [int (i) for i in train_labels]
# train_images = np.moveaxis(train_images, -1, 0)
# train_images=np.moveaxis(train_images, -1,1)
train_images = train_images.astype('float64')

extra_images = extra_images.astype('float64')
extra_labels = [int (i) for i in extra_labels]
# test_images = np.moveaxis(test_images, -1, 0)
# test_images = np.moveaxis(test_images, -1, 1)
test_images = test_images.astype('float64')
test_labels = [int (i) for i in test_labels]
train_labels=np.asarray(train_labels)
extra_labels=np.asarray(extra_labels)
train_images=np.concatenate((train_images,test_images))
train_labels=np.concatenate((train_labels,test_labels))
train_labels=train_labels.tolist()


In [3]:
# test_raw = loadmat('/content/drive/My Drive/AML_ASSIGNMENT1/test_32x32.mat')
# train_raw = loadmat('/content/drive/My Drive/AML_ASSIGNMENT1/train_32x32.mat')

In [4]:
# train_images = np.array(train_raw['X'])
# test_images = np.array(test_raw['X'])

# train_labels = train_raw['y']
# test_labels = test_raw['y']

# train_labels= [int (i) for i in train_labels]
# test_labels = [int (i) for i in test_labels]

# train_images = np.moveaxis(train_images, -1, 0)
# test_images = np.moveaxis(test_images, -1, 0)
# train_images=np.moveaxis(train_images, -1,1)
# test_images = np.moveaxis(test_images, -1, 1)

# train_images = train_images.astype('float64')
# test_images = test_images.astype('float64')

In [19]:
print(train_images.shape)
print(test_images.shape)
print(train.shape)
# print(labels.shape)
# labels=labels.tolist()
print(len(labels))
print(labels[0])
print(train_labels[0])
print(test_labels[0])

(73257, 3, 32, 32)
(26032, 3, 32, 32)
(99289, 3, 32, 32)
99289
1
1
5


In [ ]:
print(train_labels[147])

2


In [5]:
def augmentation(x):
  c, h, w = x.shape
  x += np.random.randn(c, h, w) * 0.15
  return x

def augment(x):
  beta = np.random.beta(0.5, 0.5)
  mix = max(beta, 1-beta)
  xmix = x * mix + x[::-1] * (1 - mix)
  return xmix

In [7]:
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels,
                                                  test_size=0.1, random_state=22)

In [8]:
c=[ 0 for i in range(10)]
X_train_label=[]
X_train_unlabel=[[]]
for (x, y) in zip(X_train,y_train):
  if y==10:
    y=0
  if c[y]<400:
    c[y]=c[y]+1
    X_train_label.append([x , y])
  else:
    # print(type(x))
    X_train_unlabel.append([x, augment(x)])


In [9]:
c=[ 0 for i in range(10)]
X_validation=[]
for (x, y) in zip(X_val,y_val):
  if y==10:
    y=0
  X_validation.append([x , y])

In [10]:
c=[ 0 for i in range(11)]
X_test=[]
for (x, y) in zip(test_images,test_labels):
  if y==10:
    y=0
  X_test.append([x , y])

In [11]:
labeled_trainloader=torch.utils.data.DataLoader(dataset=X_train_label, batch_size=64, shuffle=True)
unlabeled_trainloader=torch.utils.data.DataLoader(dataset=X_train_unlabel, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=X_validation, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=X_test, batch_size=64, shuffle=True)

In [12]:
print(type(labeled_trainloader))


<class 'torch.utils.data.dataloader.DataLoader'>


In [ ]:
best_accuracy=93
Train_Loss_X=[]
Train_Loss_U=[]
Train_Loss=[]
Test_Loss=[]
Validation_Loss=[]
Train_Accuracy=[]
Test_Accuracy=[]
Validation_Accuracy=[]
model = WideResNet(num_classes=10).cuda()
ema_model = WideResNet(num_classes=10).cuda()
for param in ema_model.parameters():
  param.detach_()
train_criterion = SemiLoss()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)
ema_optimizer= WeightEMA(model, ema_model, alpha=0.000002)
for epoch in range(1024):
  train_loss, train_loss_x, train_loss_u = train(labeled_trainloader, unlabeled_trainloader, model, optimizer, ema_optimizer, train_criterion,epoch)
  _, train_acc = testing(labeled_trainloader, ema_model, criterion, epoch)
  val_loss, val_acc = testing(val_loader, ema_model, criterion, epoch)
  test_loss, test_acc = testing(test_loader, ema_model, criterion, epoch)

  print("Epoch : ",epoch)
  print('Train_loss =', train_loss)
  print('Valid_loss =', val_loss)
  print('Test_loss =', test_loss)
  print('Train_accyracy =', train_acc)
  print('Val_accuracy =', val_acc)
  print('Test_accuracy =', test_acc)
  Train_Loss.append(train_loss)
  Train_Loss_X.append(train_loss_x)
  Train_Loss_U.append(train_loss_u)
  Test_Loss.append(test_loss)
  Validation_Loss.append(val_loss)
  Train_Accuracy.append(train_acc)
  Test_Accuracy.append(test_acc)
  Validation_Accuracy.append(val_acc)
  if test_acc > best_accuracy:
    best_accuracy=test_acc
    torch.save(model.state_dict(),'/content/drive/My Drive/AML_ASSIGNMENT1/model_4000_extra.pth')
    torch.save(ema_model.state_dict(),'/content/drive/My Drive/AML_ASSIGNMENT1/ema_model_4000_extra.pth')
  if epoch % 10==0:

    with open('/content/drive/My Drive/AML_ASSIGNMENT1/Train_Loss_4000_extra.txt','wb' )as fp:
        pickle.dump(Train_Loss,fp)
    with open('/content/drive/My Drive/AML_ASSIGNMENT1/Train_Loss_X_4000_extra.txt','wb') as fp:
        pickle.dump(Train_Loss_X,fp)
    with open('/content/drive/My Drive/AML_ASSIGNMENT1/Train_Loss_U_4000_extra.txt','wb') as fp:
        pickle.dump(Train_Loss_U,fp)
    with open('/content/drive/My Drive/AML_ASSIGNMENT1/Train_Accuracy_4000_extra.txt','wb' )as fp:
        pickle.dump(Train_Accuracy,fp)
    with open('/content/drive/My Drive/AML_ASSIGNMENT1/Test_Loss_4000_extra.txt','wb') as fp:
        pickle.dump(Test_Loss,fp)
    with open('/content/drive/My Drive/AML_ASSIGNMENT1/Validation_Loss_4000_extra.txt','wb') as fp:
        pickle.dump(Validation_Loss,fp)
    with open('/content/drive/My Drive/AML_ASSIGNMENT1/Test_Accuracy_4000_extra.txt','wb') as fp:
        pickle.dump(Test_Accuracy,fp)
    with open('/content/drive/My Drive/AML_ASSIGNMENT1/Validation_Accuracy_4000_extra.txt','wb') as fp:
        pickle.dump(Validation_Accuracy,fp)

Epoch :  0
Train_loss = 1.4586808276986043
Valid_loss = 1.7563951077306332
Test_loss = 1.7794467657712902
Train_accyracy = 64.925
Val_accuracy = 60.783510778303715
Test_accuracy = 60.222034416831775
Epoch :  1
Train_loss = 1.074326799126474
Valid_loss = 0.6128626386226932
Test_loss = 0.6507104116857748
Train_accyracy = 98.15
Val_accuracy = 90.67704067704068
Test_accuracy = 90.33497233704401
Epoch :  2
Train_loss = 0.9626662442093994
Valid_loss = 0.4139634524900471
Test_loss = 0.4470965940855585
Train_accyracy = 99.85
Val_accuracy = 91.8236418132277
Test_accuracy = 91.18392746449982
Epoch :  3
Train_loss = 0.9411122881492057
Valid_loss = 0.37752586639776864
Test_loss = 0.4080889942496557
Train_accyracy = 99.925
Val_accuracy = 91.91919189836365
Test_accuracy = 91.11478179762828
Epoch :  4
Train_loss = 0.9110702503413901
Valid_loss = 0.3717679278368251
Test_loss = 0.39954062177128197
Train_accyracy = 99.975
Val_accuracy = 91.86459184376359
Test_accuracy = 91.08020896888175
Epoch :  5
Trai

In [ ]:
with open('/content/drive/My Drive/AML_ASSIGNMENT1/Train_Loss_250.txt','wb' )as fp:
    pickle.dump(Train_Loss,fp)
# with open('/content/drive/My Drive/AML_ASSIGNMENT1/Train_Loss_X_250.txt','wb') as fp:
#     pickle.dump(Train_Loss_X,fp)
# with open('/content/drive/My Drive/AML_ASSIGNMENT1/Train_Loss_U_250.txt','wb') as fp:
#     pickle.dump(Train_Loss_U,fp)
# with open('/content/drive/My Drive/AML_ASSIGNMENT1/Train_Accuracy_250.txt','wb' )as fp:
#     pickle.dump(Train_Accuracy,fp)
# with open('/content/drive/My Drive/AML_ASSIGNMENT1/Test_Loss_250.txt','wb') as fp:
#     pickle.dump(Test_Loss,fp)
# with open('/content/drive/My Drive/AML_ASSIGNMENT1/Validation_Loss_250.txt','wb') as fp:
#     pickle.dump(Validation_Loss,fp)
# with open('/content/drive/My Drive/AML_ASSIGNMENT1/Test_Accuracy_250.txt','wb') as fp:
#     pickle.dump(Test_Accuracy,fp)
# with open('/content/drive/My Drive/AML_ASSIGNMENT1/Validation_Accuracy_250.txt','wb') as fp:
#     pickle.dump(Validation_Accuracy,fp)

In [12]:
def train(labeled_trainloader, unlabeled_trainloader, model, optimizer, ema_optimizer, criterion, epoch):

    losses = AverageMeter()
    losses_x = AverageMeter()
    losses_u = AverageMeter()
    ws = AverageMeter()
    labeled_train_iter = iter(labeled_trainloader)
    unlabeled_train_iter = iter(unlabeled_trainloader)
    model.train()
    
    for batch_idx in range(1024):
        try:
            inputs_x, targets_x = labeled_train_iter.next()
        except:
            labeled_train_iter = iter(labeled_trainloader)
            inputs_x, targets_x = labeled_train_iter.next()

        try:
            (inputs_u, inputs_u2)= unlabeled_train_iter.next()
        except:
            unlabeled_train_iter = iter(unlabeled_trainloader)
            (inputs_u, inputs_u2) = unlabeled_train_iter.next()


        batch_size = inputs_x.size(0)
        targets_x = torch.zeros(batch_size, 10).scatter_(1, targets_x.view(-1,1).long(), 1)

        inputs_x, targets_x = inputs_x.cuda(), targets_x.cuda(non_blocking=True)
        inputs_u = inputs_u.cuda()
        inputs_u2 = inputs_u2.cuda()

        inputs_u=inputs_u.float()
        inputs_u2=inputs_u2.float()
        inputs_x=inputs_x.float()
        with torch.no_grad():
            outputs_u = model(inputs_u)
            outputs_u2 = model(inputs_u2)
            p = (torch.softmax(outputs_u, dim=1) + torch.softmax(outputs_u2, dim=1)) / 2
            pt = p**(1/0.5)
            targets_u = pt / pt.sum(dim=1, keepdim=True)
            targets_u = targets_u.detach()

        # mixup
        all_inputs = torch.cat([inputs_x, inputs_u, inputs_u2], dim=0)
        all_targets = torch.cat([targets_x, targets_u, targets_u], dim=0)

        l = np.random.beta(0.25, 0.25)

        l = max(l, 1-l)

        idx = torch.randperm(all_inputs.size(0))

        input_a, input_b = all_inputs, all_inputs[idx]
        target_a, target_b = all_targets, all_targets[idx]

        mixed_input = l * input_a + (1 - l) * input_b
        mixed_target = l * target_a + (1 - l) * target_b

        # interleave labeled and unlabed samples between batches to get correct batchnorm calculation 
        mixed_input = list(torch.split(mixed_input, batch_size))
        mixed_input = interleave(mixed_input, batch_size)

        logits = [model(mixed_input[0])]
        for input in mixed_input[1:]:
            logits.append(model(input))

        # put interleaved samples back
        logits = interleave(logits, batch_size)
        logits_x = logits[0]
        logits_u = torch.cat(logits[1:], dim=0)

        Lx, Lu, w = criterion(logits_x, mixed_target[:batch_size], logits_u, mixed_target[batch_size:], epoch+batch_idx/1024)

        loss = Lx + w * Lu

        # record loss
        losses.update(loss.item(), inputs_x.size(0))
        losses_x.update(Lx.item(), inputs_x.size(0))
        losses_u.update(Lu.item(), inputs_x.size(0))
        ws.update(w, inputs_x.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        ema_optimizer.step()

    return (losses.avg, losses_x.avg, losses_u.avg,)

def testing(dataloader, model, criterion, epoch):

    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            inputs, targets = inputs.cuda(), targets.cuda(non_blocking=True)
            # compute output
            inputs=inputs.float()
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # measure accuracy and record loss
            prec1, prec5 = accuracy(outputs, targets, topk=(1, 5))
            losses.update(loss.item(), inputs.size(0))
            top1.update(prec1.item(), inputs.size(0))
            top5.update(prec5.item(), inputs.size(0))

    return (losses.avg, top1.avg)


def linear_rampup(current, rampup_length=1024):
    if rampup_length == 0:
        return 1.0
    else:
        current = np.clip(current / rampup_length, 0.0, 1.0)
        return float(current)

class SemiLoss(object):
    def __call__(self, outputs_x, targets_x, outputs_u, targets_u, epoch):
        probs_u = torch.softmax(outputs_u, dim=1)

        Lx = -torch.mean(torch.sum(F.log_softmax(outputs_x, dim=1) * targets_x, dim=1))
        Lu = torch.mean((probs_u - targets_u)**2)

        return Lx, Lu, 250* linear_rampup(epoch)

class WeightEMA(object):
    def __init__(self, model, ema_model, alpha=0.999):
        self.model = model
        self.ema_model = ema_model
        self.alpha = alpha
        self.params = list(model.state_dict().values())
        self.ema_params = list(ema_model.state_dict().values())
        self.wd = 0.02 * 0.002

        for param, ema_param in zip(self.params, self.ema_params):
            param.data.copy_(ema_param.data)

    def step(self):
        one_minus_alpha = 1.0 - self.alpha
        for param, ema_param in zip(self.params, self.ema_params):
            if ema_param.dtype==torch.float32:
                ema_param.mul_(self.alpha)
                ema_param.add_(param * one_minus_alpha)
                # customized weight decay
                param.mul_(1 - self.wd)

def interleave_offsets(batch, nu):
    groups = [batch // (nu + 1)] * (nu + 1)
    for x in range(batch - sum(groups)):
        groups[-x - 1] += 1
    offsets = [0]
    for g in groups:
        offsets.append(offsets[-1] + g)
    assert offsets[-1] == batch
    return offsets


def interleave(xy, batch):
    nu = len(xy) - 1
    offsets = interleave_offsets(batch, nu)
    xy = [[v[offsets[p]:offsets[p + 1]] for p in range(nu + 1)] for v in xy]
    for i in range(1, nu + 1):
        xy[0][i], xy[i][i] = xy[i][i], xy[0][i]
    return [torch.cat(v, dim=0) for v in xy]


class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def accuracy(output, target, topk=(1,)):
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [13]:

class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride, dropRate=0.0, activate_before_residual=False):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes, momentum=0.001)
        self.relu1 = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes, momentum=0.001)
        self.relu2 = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate
        self.equalInOut = (in_planes == out_planes)
        self.convShortcut = (not self.equalInOut) and nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride,
                               padding=0, bias=False) or None
        self.activate_before_residual = activate_before_residual
    def forward(self, x):
        if not self.equalInOut and self.activate_before_residual == True:
            x = self.relu1(self.bn1(x))
        else:
            out = self.relu1(self.bn1(x))
        out = self.relu2(self.bn2(self.conv1(out if self.equalInOut else x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, training=self.training)
        out = self.conv2(out)
        return torch.add(x if self.equalInOut else self.convShortcut(x), out)

class NetworkBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, out_planes, block, stride, dropRate=0.0, activate_before_residual=False):
        super(NetworkBlock, self).__init__()
        self.layer = self._make_layer(block, in_planes, out_planes, nb_layers, stride, dropRate, activate_before_residual)
    def _make_layer(self, block, in_planes, out_planes, nb_layers, stride, dropRate, activate_before_residual):
        layers = []
        for i in range(int(nb_layers)):
            layers.append(block(i == 0 and in_planes or out_planes, out_planes, i == 0 and stride or 1, dropRate, activate_before_residual))
        return nn.Sequential(*layers)
    def forward(self, x):
        return self.layer(x)

class WideResNet(nn.Module):
    def __init__(self, num_classes, depth=28, widen_factor=2, dropRate=0.0):
        super(WideResNet, self).__init__()
        nChannels = [16, 16*widen_factor, 32*widen_factor, 64*widen_factor]
        assert((depth - 4) % 6 == 0)
        n = (depth - 4) / 6
        block = BasicBlock
        # 1st conv before any network block
        self.conv1 = nn.Conv2d(3, nChannels[0], kernel_size=3, stride=1,
                               padding=1, bias=False)
        # 1st block
        self.block1 = NetworkBlock(n, nChannels[0], nChannels[1], block, 1, dropRate, activate_before_residual=True)
        # 2nd block
        self.block2 = NetworkBlock(n, nChannels[1], nChannels[2], block, 2, dropRate)
        # 3rd block
        self.block3 = NetworkBlock(n, nChannels[2], nChannels[3], block, 2, dropRate)
        # global average pooling and classifier
        self.bn1 = nn.BatchNorm2d(nChannels[3], momentum=0.001)
        self.relu = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.fc = nn.Linear(nChannels[3], num_classes)
        self.nChannels = nChannels[3]

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                m.bias.data.zero_()

    def forward(self, x):
        out = self.conv1(x)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.relu(self.bn1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(-1, self.nChannels)
        return self.fc(out)